<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [3]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip -q install pdfx
  import pdfx

     |████████████████████████████████| 5.6MB 13.8MB/s 
     |████████████████████████████████| 184kB 43.4MB/s 
     |████████████████████████████████| 3.2MB 39.0MB/s 
ERROR: requests 2.23.0 has requirement chardet<4,>=3.0.2, but you'll have chardet 4.0.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [5]:
!pip -q install xlsxwriter

     |████████████████████████████████| 153kB 8.9MB/s 


In [6]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [7]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [8]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [9]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 4 15


In [10]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading DOH Data Drop...')
      downloaded = True
  else:
    continue

In [11]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [12]:
landline_masterlist = landline.make_masterlist()

In [13]:
# hospital daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 05 DOH Data Collect - Daily Report.csv'
# hospital weekly inventory of supplies
sup_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 06 DOH Data Collect - Weekly Report.csv'
print('weekly supplies inventory:', sup_filename)
print('daily capability count:', cap_filename)
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

weekly supplies inventory: DOH COVID Data Drop_ 20210415 - 06 DOH Data Collect - Weekly Report.csv
daily capability count: DOH COVID Data Drop_ 20210415 - 05 DOH Data Collect - Daily Report.csv


In [14]:
#load capacity data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])
df_cap.drop(['updateddate', 'addeddate', 'reportdate', 'id', 'north_coord', 'east_coord'], axis=1, inplace=True)
df_cap[hosp_code] = df_cap[hosp_code].apply(lambda x: x.strip())

In [15]:
!gdown -q 'https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json'

In [16]:
df_nhfr = pd.read_json('nhfr.json')
print('Refer to https://nhfr.doh.gov.ph/ for the latest data')

Refer to https://nhfr.doh.gov.ph/ for the latest data


In [17]:
df_cap = df_cap.merge(how='left', right=df_nhfr, left_on='hfhudcode', right_on='Health Facility Code')

In [18]:
#load supplies data
#df_sup = pd.read_csv(sup_filename)
#df_sup['sup_reportdate'] = pd.to_datetime(df_sup['reportdate'])
#df_sup.drop(['updateddate', 'addeddate', 'reportdate', 'id'], axis=1, inplace=True)
#df_sup[hosp_code] = df_sup[hosp_code].apply(lambda x: x.strip())

In [19]:
# quarantine facility daily patient census
#quarantine_daily = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 08 Quarantine Facility Data - Daily Report.csv')
# quarantine facility weekly inventory of supplies
#quarantine_weekly = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 09 Quarantine Facility Data - Weekly Report.csv')

In [20]:
# HCW availability
#baseline_v3 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 10 DOH Data Collect v3 - Baseline.csv')
# mechvent, swabbing, ipc
#baseline_v4 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 11 DOH Data Collect v4 - Baseline.csv')
# quarantine and isolation facilities
#ttmf = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 12 DDC TTMF Tracker v1 - Baseline.csv')

In [21]:
#print('{0}\n{1}'.format(df_cap.keys(), df_sup.keys()))
#print('{0}\n{1}'.format(quarantine_weekly.keys(), quarantine_daily.keys()))

In [22]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [23]:
#prefix = ['ics', 'resdoc', 'nurse', 'medtech', 'respthe', 'radtech', 'support']
#suffix = ['_total', '_er', '_icu', '_ward'] 
#triage = ['resdoc_triage', 'nurse_triage', 'support_triage']
#added = ['Augmen_total']

In [24]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [25]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.mechvent_v > 0) & \
    (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.icu_v > 0) & \
    (df_cap.icu_o > 0)
  df_vacant = df_cap[filter][['cap_reportdate', 'cfname', 'city_mun', 'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                            'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                            'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [26]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,6516,2021-04-12,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,27,162,52,0,23,288,8,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,152860,2021-04-12,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,12,18,10,101,5,47,3,64,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
2,272418,2021-04-12,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,7,13,9,0,3,38,6,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
3,154292,2021-04-12,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,6,42,1,33,4,14,3,51,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
4,293685,2021-04-12,EAST AVENUE MEDICAL CENTER,QUEZON CITY,5,15,30,0,55,205,54,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
5,230531,2021-04-12,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,5,0,1,3,5,12,14,13,(02) 8716 1853,None,(02) 8716 1853,Private
6,52577,2021-04-12,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,4,0,1,0,25,94,3,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
7,13128,2021-04-12,MARIKINA VALLEY MEDICAL CENTER,CITY OF MARIKINA,2,3,5,0,4,25,1,0,(02) 8682 2222,(02) 8681 3398,(02) 8682 3399,Private
8,20259,2021-04-12,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,2,8,2,11,5,10,3,8,(02) 8723 0041,(02) 8893 5762,None,Private
9,299952,2021-04-12,NATIONAL KIDNEY AND TRANSPLANT INSTITUTE,QUEZON CITY,2,24,12,1,5,89,13,29,(02) 8981 0330,(02) 8924 0701,(02) 8922 5608,Government


In [27]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,152861,2021-04-13,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,12,12,10,79,5,53,3,82,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
1,6517,2021-04-13,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,7,181,56,0,43,269,4,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
2,272419,2021-04-13,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,5,10,5,0,10,26,10,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
3,200250,2021-04-13,MANILA DOCTORS HOSPITAL,ERMITA,4,26,11,0,20,44,13,0,(02) 8558 0888,None,(02) 8524 7376,Private
4,230532,2021-04-13,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,4,2,2,3,6,9,12,13,(02) 8716 1853,None,(02) 8716 1853,Private
5,154293,2021-04-13,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,4,45,1,27,6,15,2,57,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
6,293686,2021-04-13,EAST AVENUE MEDICAL CENTER,QUEZON CITY,4,37,30,0,56,176,50,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
7,20260,2021-04-13,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,3,8,2,11,5,10,4,8,(02) 8723 0041,(02) 8893 5762,None,Private
8,55822,2021-04-13,UP-PHILIPPINE GENERAL HOSPITAL,ERMITA,3,28,15,6,25,78,55,129,(02) 8554 8450,(02) 8523 7123,(02) 8524 2211,Government
9,187422,2021-04-13,RIZAL MEDICAL CENTER,CITY OF PASIG,3,26,19,0,15,89,3,0,(02) 8865 8400,(02) 8671 9616,(02) 8671 9616,Government


In [28]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,272420,2021-04-14,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,18,23,5,0,2,34,10,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
1,152862,2021-04-14,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,14,12,10,76,5,53,3,87,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
2,6518,2021-04-14,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,10,208,53,0,40,242,7,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
3,52579,2021-04-14,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,6,3,4,0,23,84,0,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
4,293687,2021-04-14,EAST AVENUE MEDICAL CENTER,QUEZON CITY,5,57,26,0,55,163,56,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
5,200251,2021-04-14,MANILA DOCTORS HOSPITAL,ERMITA,4,25,10,0,20,45,14,0,(02) 8558 0888,None,(02) 8524 7376,Private
6,230533,2021-04-14,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,4,2,4,4,6,9,9,12,(02) 8716 1853,None,(02) 8716 1853,Private
7,252477,2021-04-14,TONDO MEDICAL CENTER,TONDO I / II,3,33,3,0,15,5,3,88,(02) 8522 1174,(02) 8522 9244,(02) 8252 8661,Government
8,4805,2021-04-14,UNIVERSITY OF PERPETUAL HELP DALTA MEDICAL CEN...,CITY OF LAS PIÑAS,3,0,7,0,11,38,11,0,(02) 8874 8515,(02) 8873 7210,(02) 8873 7210,Private
9,299954,2021-04-14,NATIONAL KIDNEY AND TRANSPLANT INSTITUTE,QUEZON CITY,3,9,17,0,4,81,8,30,(02) 8981 0330,(02) 8924 0701,(02) 8922 5608,Government


In [29]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['II'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,172584,2021-04-14,"DIVINE MERCY WELLNESS CENTER, INC.",TUGUEGARAO CITY (CAPITAL),1,2,2,0,2,12,2,3,+63 788444624,None,None,Private


In [30]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['IV-A'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,274127,2021-04-14,DE LA SALLE UNIVERSITY MEDICAL CENTER,CITY OF DASMARIÑAS,5,11,7,0,3,46,16,0,(02) 8988 3100,+63 464818000,None,Private
1,220532,2021-04-14,GENERAL EMILIO AGUINALDO MEMORIAL HOSPITAL,TRECE MARTIRES CITY (CAPITAL),4,17,10,12,1,7,0,3,+63 464190063,None,None,Government
2,211688,2021-04-14,"GLOBAL CARE MEDICAL CENTER OF CANLUBANG, INC.",CITY OF CALAMBA,3,1,3,1,2,0,0,15,+63 495205626,None,None,Private
3,228528,2021-04-14,QUEZON MEDICAL CENTER,LUCENA CITY (CAPITAL),3,8,7,17,4,30,3,55,+63 427104023,None,None,Government
4,344593,2021-04-14,"ST. JAMES HOSPITAL, INC.",CITY OF SANTA ROSA,3,0,3,0,1,4,0,0,+63 495341254,None,None,Private
5,94261,2021-04-14,MEDICAL CENTER WESTERN BATANGAS,BALAYAN,2,3,1,3,5,2,3,3,+63 434071103,None,+63 434071101,Private
6,111435,2021-04-14,UNIHEALTH-SOUTHWOODS HOSPITAL AND MEDICAL CENT...,CITY OF BIÑAN,2,17,15,8,3,4,0,7,+63 498616385,None,None,Private
7,298808,2021-04-14,DANIEL O. MERCADO MEDICAL CENTER,CITY OF TANAUAN,2,9,2,0,2,1,0,15,+63 437781810,+63 437780960,None,Private
8,182006,2021-04-14,"ST. FRANCES CABRINI MEDICAL CENTER, INC.",SANTO TOMAS,1,4,4,0,5,4,2,7,+63 487784811,None,+63 487784819,Private
9,251332,2021-04-14,"LOS BA?OS DOCTORS HOSPITAL AND MEDICAL CENTER,...",LOS BAÑOS,1,1,2,0,4,20,2,0,+63 495360100,None,+63 495361225,Private


In [31]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['III'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,215119,2021-04-14,BULACAN MEDICAL CENTER,CITY OF MALOLOS (CAPITAL),12,69,54,24,18,18,1,76,+63 447910630,None,None,Government
1,3144,2021-04-14,BATAAN GENERAL HOSPITAL AND MEDICAL CENTER,CITY OF BALANGA (CAPITAL),10,33,22,22,35,75,26,9,+63 472372269,None,None,Government
2,13688,2021-04-14,JOSE B. LINGAD MEMORIAL REGIONAL HOSPITAL,CITY OF SAN FERNANDO (CAPITAL),4,0,17,14,78,0,11,285,+63 459613544,None,None,Government
3,260479,2021-04-14,"GRACE GENERAL HOSPITAL, INC.",CITY OF SAN JOSE DEL MONTE,3,5,1,0,1,9,0,0,+63 445194838,None,None,Private
4,81486,2021-04-14,"SACRED HEART HOSPITAL OF MALOLOS, INC.",CITY OF MALOLOS (CAPITAL),2,3,6,0,4,14,2,0,+63 447940561,None,+63 447912911,Private
5,132578,2021-04-14,ALLIED CARE EXPERTS (ACE) MEDICAL CENTER ?BALI...,BALIUAG,2,7,1,0,4,23,3,0,+63 448161000,None,+63 448161000,Private
6,358903,2021-04-14,JAMES L. GORDON MEMORIAL HOSPITAL,OLONGAPO CITY,2,35,4,0,2,34,2,8,+63 476021229,+63 476021230,+63 476021228,Government
7,21694,2021-04-14,PREMIERE MEDICAL CENTER,CABANATUAN CITY,1,18,2,5,3,2,0,16,+63 444637845,None,None,Private
8,79194,2021-04-14,"NUEVA ECIJA DOCTORS? HOSPITAL, INC.",CABANATUAN CITY,1,18,2,5,2,10,1,4,+63 449400176,None,None,Private
9,305394,2021-04-14,"THE MEDICAL CITY CLARK, INC.",MABALACAT CITY,1,13,1,2,1,21,1,2,+63 453008888,None,None,Private


In [32]:
roi = reg_lookup['NCR']
#roi = reg_lookup['II']
#roi = reg_lookup['IV-A']
filter = (df_cap.cap_reportdate == day_ago_from(1, data_date)) & (df_cap.region == roi)
occupied = df_cap[filter]['icu_o'].sum()
vacant = df_cap[filter]['icu_v'].sum()
pct_used = (occupied / (vacant + occupied)) * 100

In [33]:
def dump_daily(df_cap, days_ago):
  filter = (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) 
  df_vacant = df_cap[filter][['cfname', 'region', 'province', 'city_mun', 
                              'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                              'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                              'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True, drop=True)
  return df_vacant

In [34]:
#https://stackoverflow.com/questions/29463274/simulate-autofit-column-in-xslxwriter
def get_col_widths(dataframe):
    # First we find the maximum length of the index column   
    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    # Then, we concatenate this to the max of the lengths of column name and its values for each column, left to right
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]

In [35]:
writer = pd.ExcelWriter('hospitals_DOHDataDrop_' + data_date.strftime('%Y%m%d') + '.xlsx', engine='xlsxwriter')
workbook  = writer.book
format_wrap = workbook.add_format({'text_wrap': True})

In [36]:
readme = 'README'
notes = []
notes.append('Data source: DOH Data Drop ' + data_date.strftime('%Y-%m-%d'))
notes.append('Data source link is in the README PDF @ ' + 'https://bit.ly/DataDropPH')
notes.append('Info here is dated -- call the ONE HOSPITAL COMMAND CENTER for current availability. Reach them through the ff. numbers: (02) 8865-0500, 0915-777-7777 and 0919-977-3333')
notes.append('cap_reportdate = date of report for icu, isolation rooms, beds in ward, ventilators')
notes.append('IMPORTANT: Please check https://nhfr.doh.gov.ph/rfacilities2list.php for latest hospital info')
notes.append('Default sorting is by number of vacant ICU beds - largest to smallest.')
notes.append('This is not an official document. It has not been validated - there could be errors.')
notes.append('Refer to ncovtracker.doh.gov.ph for official announcements')
notes.append('Very important: Please contact the ONE HOSPITAL COMMAND CENTER if you are looking for a hospital room / healthcare referral.')
notes.append('OHCC contact: (02) 8865-0500, 0915-777-7777 and 0919-977-3333')
notes.append('DOH COVID-19 toll-free hotline number: 1555')
notes.append('Source code at https://github.com/maytlim/doh_data')
df_readme = pd.DataFrame(notes)
df_readme.to_excel(writer, readme)

In [37]:
for days_ago in range(1, 4):
  reportdate = day_ago_from(days_ago, data_date)
  df_dumpdaily = dump_daily(df_cap, days_ago)
  df_dumpdaily.to_excel(writer, reportdate)
  worksheet = writer.sheets[reportdate]
  worksheet.set_row(0, None, cell_format=format_wrap)
  for i, width in enumerate(get_col_widths(df_dumpdaily)):
    worksheet.set_column(i, i, min(20, max(width + 4, 5)))
  (max_row, max_col) = df_dumpdaily.shape
  # Set the autofilter
  worksheet.autofilter(0, 1, max_row, 12)

In [38]:
writer.close()